In [1]:
%%writefile mapper1.py

# Your code for mapper here.
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

path = 'stop_words_en.txt'
def read_stop_words(file_path):
    return set(word.strip().lower() for word in open(file_path))
stop_words = read_stop_words(path)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word.lower() not in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
            print "%s\t%d" % (word.lower(), 1)

Writing mapper1.py


In [2]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer1.py


In [3]:
%%writefile mapper2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        if word.isalpha():
            word_permuted = ''.join(sorted(word))
            print "%s\t%s\t%d" % (word_permuted, word, count)
    except ValueError as e:
        continue

Writing mapper2.py


In [4]:
%%writefile reducer2.py

# Your code for reducer here.
import sys

current_key = None
permuted_sum = 0
word_group = []

for line in sys.stdin:
    try:
        key, word, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            if len(word_group) > 1:
                word_string = ','.join(sorted(word_group))
                print "%d\t%d\t%s" % (permuted_sum, len(word_group), word_string)
        current_key = key
        permuted_sum = 0
        word_group = []
    permuted_sum += count
    word_group.append(word)

if current_key:
    if len(word_group) > 1:
        word_string = ','.join(sorted(word_group))
        print "%d\t%d\t%s" % (permuted_sum, len(word_group), word_string)

Writing reducer2.py


In [5]:
%%bash

OUT_DIR="wordgroup_wordcount_"$(date +"%s%6N")

NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR1="wordgroup_permutation_"$(date +"%s%6N")

NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming permutations" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR1} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR1}/part-00000 | grep -P '(,|\t)english($|,)'
hdfs dfs -rm -r -skipTrash ${OUT_DIR1}* > /dev/null

7820	5	english,helsing,hesling,shengli,shingle


rm: `wordgroup_wordcount_1534566984903939': No such file or directory
18/08/18 04:36:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/18 04:36:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/18 04:36:30 INFO mapred.FileInputFormat: Total input files to process : 1
18/08/18 04:36:30 INFO mapreduce.JobSubmitter: number of splits:2
18/08/18 04:36:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1534520873842_0001
18/08/18 04:36:31 INFO impl.YarnClientImpl: Submitted application application_1534520873842_0001
18/08/18 04:36:31 INFO mapreduce.Job: The url to track the job: http://2ce61352a0f0:8088/proxy/application_1534520873842_0001/
18/08/18 04:36:31 INFO mapreduce.Job: Running job: job_1534520873842_0001
18/08/18 04:36:38 INFO mapreduce.Job: Job job_1534520873842_0001 running in uber mode : false
18/08/18 04:36:38 INFO mapreduce.Job:  map 0% reduce 0%
18/08/18 04:36:54 INFO mapreduce.Job:  map 38% reduce 0%
18/08/18 0